In [4]:
from vkprog_data_prep import load_booking_data
from pa_lib.data import desc_col

In [16]:
bd = load_booking_data()

2019-10-08 15:59:03 [INFO] Started loading binary file
2019-10-08 15:59:03 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-10-08 15:59:03 [INFO] Finished loading binary file in 0.17s (0.8s CPU)


In [17]:
bd.head(3)

,Endkunde_NR,Endkunde,EK_Abc,EK_Boni,EK_Plz,EK_Ort,EK_Land,EK_HB_Apg_Kurzz,EK_Kam_Betreut,EK_Aktiv,Agentur,AG_Hauptbetreuer,Verkaufsberater,Endkunde_Branchengruppe_ID,Endkunde_Branchengruppe,Endkunde_Nbranchengruppe_ID,Endkunde_Nbranchengruppe,Endkunde_Branchenkat_ID,Endkunde_Branchenkat,Endkunde_Nbranchenkat_ID,Endkunde_Nbranchenkat,Auftrag_Branchengruppe_ID,Auftrag_Branchengruppe,Auftrag_Nbranchengruppe_ID,Auftrag_Nbranchengruppe,Auftrag_Branchenkat_ID,Auftrag_Branchenkat,Auftrag_Nbranchenkat_ID,Auftrag_Nbranchenkat,Agps_NR,Segment,KV_NR,KV_Typ,Kampagnen_Status,Kampagne_Erfassungsdatum,Kampagne_Beginn,Auftragsart,Res_Dat,Annullation_Datum,Aush_Von,Dauer,Vertrag,Brutto,Netto,Agglo,PF,Kamp_Beginn_Jahr,Kamp_Beginn_KW,Kamp_Beginn_KW_2,Kamp_Beginn_KW_4,Kamp_Erfass_Jahr,Kamp_Erfass_KW,Kamp_Erfass_KW_2,Kamp_Erfass_KW_4
2,123010,Perrodin Voyages,NaN,gut,1920,Martigny,SCHWEIZ,NaN,0,1,NaN,NaN,VRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,1035675,Traffic,540946,KDVTLG,4,2008-10-01,2009-01-01,Traffic Auftrag,2008-10-01,NaT,2009-01-01,365,Ja,1067,1067,NaN,Hecktafel,2009,1,1,1,2008,40,39,37
3,411700,"Maharaja, traiteur, cuisine indienne",E,gut,1004,Lausanne,SCHWEIZ,SAO,0,1,NaN,NaN,YTA,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,1035902,Traffic,541110,KDVTLG,4,2008-10-01,2009-01-01,Traffic Auftrag,2008-10-01,NaT,2009-01-01,365,Ja,1125,1125,NaN,"Schiebekarton,Seitenbanner,Teilheck Fenster",2009,1,1,1,2008,40,39,37
4,482093,McDonald's Suisse Restaurants Sàrl,A,gut,1023,Crissier,SCHWEIZ,BEG,1,1,OMD Schweiz AG,FER,NIE,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,1035961,APG|SGA,541161,KDVTLG,4,2008-10-01,2008-12-29,langfristiger Vertrag,2008-10-01,NaT,2008-12-29,371,Ja,5500,5500,"90768,91372,93379,A0230,A0261,A0351,A0371,A058...","Dachlängsformat,Diverse,F12,F12L,F12LR,F200,F2...",2009,1,1,1,2008,40,39,37


In [18]:
desc_col(bd.loc[:,["Endkunde_NR",
                                 "Kampagne_Erfassungsdatum",
                                 "Endkunde_Branchengruppe_ID",
                                 "Endkunde_Branchengruppe",
                                 "Auftrag_Branchengruppe_ID",
                                 "Auftrag_Branchengruppe"]])

,DTYPE,NULLS,UNIQUE
Endkunde_NR,int64,0/669906,32031
Kampagne_Erfassungsdatum,datetime64[ns],0/669906,3290
Endkunde_Branchengruppe_ID,category,5647/664259,83
Endkunde_Branchengruppe,category,5647/664259,83
Auftrag_Branchengruppe_ID,category,0/669906,83
Auftrag_Branchengruppe,category,0/669906,83


In [15]:


bd.loc[:,["Endkunde_NR",
                        "Kampagne_Erfassungsdatum",
                        "Endkunde_Branchengruppe_ID",
                        "Endkunde_Branchengruppe",
                        "Auftrag_Branchengruppe_ID",
                        "Auftrag_Branchengruppe"]] #.sort_values(["Endkunde_NR","Kampagne_Erfassungsdatum"],ascending=[True,False])

,Endkunde_NR,Kampagne_Erfassungsdatum,Endkunde_Branchengruppe_ID,Endkunde_Branchengruppe,Auftrag_Branchengruppe_ID,Auftrag_Branchengruppe
592277,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
614280,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
1178728,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
1194644,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
1199212,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
1290222,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
1314515,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
1363082,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
1363083,100034,2018-10-31,720,WG - Dienstleistung,720,WG - Dienstleistung
1002036,100034,2018-02-06,720,WG - Dienstleistung,720,WG - Dienstleistung


In [89]:
%%time
import pandas as pd


kunden_branchen_df  = bd.loc[:,["Endkunde_NR",
                                "Kampagne_Erfassungsdatum",
                                "Endkunde_Branchengruppe_ID"]
                            ]

auftrag_branchen_df = bd.loc[:,["Endkunde_NR",
                                "Kampagne_Erfassungsdatum",
                                "Auftrag_Branchengruppe_ID"]
                            ]

kunden_branchen_df.columns  = ["Endkunde_NR","Kampagne_Erfassungsdatum","Branchen_ID"]
auftrag_branchen_df.columns = ["Endkunde_NR","Kampagne_Erfassungsdatum","Branchen_ID"]

branchen_df = (kunden_branchen_df.append(auftrag_branchen_df)
                                 .dropna()
                                 .drop_duplicates()
              )

branchen_df = (branchen_df
               .loc[:,:]
               .pivot_table(
                    index      = ["Endkunde_NR"],
                    columns    = ["Branchen_ID"],
                    values     = ["Branchen_ID"],
                    aggfunc    = "count",
                    fill_value = 0
                           )
              )

branchen_df = pd.DataFrame(branchen_df.to_records(index=True))
new_col_names = [x.replace("('Kampagne_Erfassungsdatum',","B").replace(" '","").replace("')","") for x in branchen_df.columns]
branchen_df.columns = new_col_names

Wall time: 14.7 s


In [90]:
branchen_df.head(4)

,Endkunde_NR,B105,B110,B115,B116,B120,B125,B130,B135,B140,B145,B150,B155,B160,B165,B170,B185,B195,B225,B230,B240,B241,B290,B295,B325,B330,B332,B334,B339,B349,B350,B351,B352,B353,B354,B356,B357,B366,B367,B371,...,B410,B420,B425,B435,B436,B445,B446,B447,B448,B455,B460,B464,B465,B466,B468,B469,B470,B475,B503,B504,B508,B509,B535,B555,B556,B575,B610,B695,B705,B720,B765,B780,B795,B800,B901,B950,B960,B970,B980,B990
0,100034,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44,0,0,0,0,0,0,0,0,0,0
1,100039,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,100061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,100064,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,9,0,1,0,0,0


In [82]:
test

['B105',
 'B110',
 'B115',
 'B116',
 'B120',
 'B125',
 'B130',
 'B135',
 'B140',
 'B145',
 'B150',
 'B155',
 'B160',
 'B165',
 'B170',
 'B185',
 'B195',
 'B225',
 'B230',
 'B240',
 'B241',
 'B290',
 'B295',
 'B325',
 'B330',
 'B332',
 'B334',
 'B339',
 'B349',
 'B350',
 'B351',
 'B352',
 'B353',
 'B354',
 'B356',
 'B357',
 'B366',
 'B367',
 'B371',
 'B390',
 'B400',
 'B401',
 'B405',
 'B410',
 'B420',
 'B425',
 'B435',
 'B436',
 'B445',
 'B446',
 'B447',
 'B448',
 'B455',
 'B460',
 'B464',
 'B465',
 'B466',
 'B468',
 'B469',
 'B470',
 'B475',
 'B503',
 'B504',
 'B508',
 'B509',
 'B535',
 'B555',
 'B556',
 'B575',
 'B610',
 'B695',
 'B705',
 'B720',
 'B765',
 'B780',
 'B795',
 'B800',
 'B901',
 'B950',
 'B960',
 'B970',
 'B980',
 'B990']